In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importing the packages
import numpy as np
import pandas as pd
#import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import KFold
from sklearn.ensemble import VotingClassifier
import warnings
warnings.simplefilter(action = "ignore")
sns.set()
plt.style.use('ggplot')
%matplotlib inline

In [117]:
df_d=pd.read_csv('/content/drive/MyDrive/Disease Prediction 2/Cleaned Datasets/diabetescleaned.csv')
df_h=pd.read_csv('/content/drive/MyDrive/Disease Prediction 2/Cleaned Datasets/heartcleaned.csv')
df_k=pd.read_csv('/content/drive/MyDrive/Disease Prediction 2/Cleaned Datasets/kidneycleaned.csv')
df_l=pd.read_csv('/content/drive/MyDrive/Disease Prediction 2/Cleaned Datasets/livercleaned.csv')

In [119]:
import pandas as pd

def drop_weak_correlations_with_target(data, target_column, threshold):
    """Drops features with correlations below `threshold` (absolute value) with respect to the target column.  Assumes target is the last column .
    Args:
        data: A pandas DataFrame.
        target_column: The name of the classification target feature.
        threshold: The correlation coefficient threshold.

    Returns:
        A new DataFrame with weakly correlated features removed.
    """
    correlations = data.corr()[target_column].abs()[:-1]  # Exclude the target itself
    weak_features = correlations[correlations < threshold].index
    return data.drop(weak_features, axis=1)

def get_feature_count(data, target_column, threshold):
    new_data = drop_weak_correlations_with_target(data.copy(), target_column, threshold)
    return len(new_data.columns) - 1  # Subtract 1 for the target column

def find_threshold(datasets, target_column, desired_feature_count):
    threshold = 0.0
    increment = 0.1

    while True:
        feature_counts = [get_feature_count(data, target_column, threshold) for data in datasets]
        min_features = min(feature_counts)
        max_features = max(feature_counts)

        if min_features == max_features == desired_feature_count:
            return threshold

        if max_features > desired_feature_count:
            threshold += increment
        else:
            threshold -= increment
            increment /= 2




In [124]:
# ... (Load your datasets df_d, df_h, df_k, and df_l)
datasets=[df_k]
target_column = "class"  # Example; adjust accordingly
desired_feature_count = 8

final_threshold = find_threshold(datasets.copy(), target_column, desired_feature_count)
print("Final Threshold:", final_threshold)

Final Threshold: 0.39375000000000004


In [125]:
# Your dataset and target column (assuming 'Outcome' is the correct target for the kidney disease dataset)
data = df_k.copy()  # Make a copy to avoid modifying your original dataset
target_column = "class"

# The threshold determined using the iterative process
final_threshold = 0.39375000000000004

# Calling the function to drop weakly correlated features
new_data_k = drop_weak_correlations_with_target(data, target_column, final_threshold)

print(new_data_k.head())  # Display the first few rows of the modified dataset

   specific_gravity  albumin  pus_cell  haemoglobin  packed_cell_volume  \
0             1.020      1.0         1         15.4                44.0   
1             1.020      4.0         1         11.3                38.0   
2             1.010      2.0         1          9.6                31.0   
3             1.005      4.0         0         11.2                32.0   
4             1.010      2.0         1         11.6                35.0   

   red_blood_cell_count  hypertension  diabetes_mellitus  class  
0                   5.2             1                  1      0  
1                   2.1             0                  0      0  
2                   5.7             0                  1      0  
3                   3.9             1                  0      0  
4                   4.6             0                  0      0  


In [126]:
# ... (Load your datasets df_d, df_h, df_k, and df_l)
datasets=[df_h]
target_column = "target"  # Example; adjust accordingly
desired_feature_count = 8

final_threshold = find_threshold(datasets.copy(), target_column, desired_feature_count)
print("Final Threshold:", final_threshold)

Final Threshold: 0.25000000000000006


In [127]:
# Your dataset and target column (assuming 'Outcome' is the correct target for the kidney disease dataset)
data = df_h.copy()  # Make a copy to avoid modifying your original dataset
target_column = "target"

# The threshold determined using the iterative process
final_threshold = 0.25000000000000006

# Calling the function to drop weakly correlated features
new_data_h = drop_weak_correlations_with_target(data, target_column, final_threshold)

print(new_data_h.head())  # Display the first few rows of the modified dataset

   sex  cp   thalach  exang  oldpeak  slope  ca  thal  target
0    1   3  5.010635      0      2.3      0   0     1       1
1    1   2  5.231109      0      3.5      0   0     2       1
2    0   1  5.147494      0      1.4      2   0     2       1
3    1   1  5.181784      0      0.8      2   0     2       1
4    0   0  5.093750      1      0.6      2   0     2       1


In [129]:
# ... (Load your datasets df_d, df_h, df_k, and df_l)
datasets=[df_d]
target_column = "Outcome"  # Example; adjust accordingly
desired_feature_count = 8

final_threshold = find_threshold(datasets.copy(), target_column, desired_feature_count)
print("Final Threshold:", final_threshold)

Final Threshold: 0.23750000000000007


In [131]:
# Your dataset and target column (assuming 'Outcome' is the correct target for the kidney disease dataset)
data = df_d.copy()  # Make a copy to avoid modifying your original dataset
target_column = "Outcome"

# The threshold determined using the iterative process
final_threshold = 0.23750000000000007

# Calling the function to drop weakly correlated features
new_data_d = drop_weak_correlations_with_target(data, target_column, final_threshold)

print(new_data_d.head())  # Display the first few rows of the modified dataset

   Glucose  SkinThickness  Insulin   BMI  Age  Outcome  \
0    148.0           35.0    169.5  33.6   50        1   
1     85.0           29.0    102.5  26.6   31        0   
2    183.0           32.0    169.5  23.3   32        1   
3     89.0           23.0     94.0  28.1   21        0   
4    137.0           35.0    168.0  43.1   33        1   

   NewInsulinScore_Normal  NewGlucose_Normal  NewGlucose_Secret  
0                       0                  0                  1  
1                       1                  1                  0  
2                       0                  0                  1  
3                       1                  1                  0  
4                       0                  0                  1  


In [133]:
# ... (Load your datasets df_d, df_h, df_k, and df_l)
datasets=[df_l]
target_column = "Result"  # Example; adjust accordingly
desired_feature_count = 8

final_threshold = find_threshold(datasets.copy(), target_column, desired_feature_count)
print("Final Threshold:", final_threshold)

Final Threshold: 0.1


In [134]:
# Your dataset and target column (assuming 'Outcome' is the correct target for the kidney disease dataset)
data = df_l.copy()  # Make a copy to avoid modifying your original dataset
target_column = "Result"

# The threshold determined using the iterative process
final_threshold = 0.1

# Calling the function to drop weakly correlated features
new_data_l = drop_weak_correlations_with_target(data, target_column, final_threshold)

print(new_data_l.head())  # Display the first few rows of the modified dataset

   Gender  Total Bilirubin  Direct Bilirubin  Alkaline_Phosphotase  \
0  Female              0.7               0.1                 187.0   
1    Male             10.9               5.5                 699.0   
2    Male              7.3               4.1                 490.0   
3    Male              1.0               0.4                 182.0   
4    Male              3.9               2.0                 195.0   

   Alamine_Aminotransferase  Aspartate_Aminotransferase  Albumin  \
0                      16.0                        18.0      3.3   
1                      64.0                       100.0      3.2   
2                      60.0                        68.0      3.3   
3                      14.0                        20.0      3.4   
4                      27.0                        59.0      2.4   

   Albumin_and_Globulin Ratio  Result  
0                        0.90       1  
1                        0.74       1  
2                        0.89       1  
3         

In [135]:
## if score==negative, mark 0 ;else 1
def partition(x):
    if x =='Male':
        return 0
    return 1

new_data_l['Gender'] = new_data_l['Gender'].map(partition)

In [140]:
new_data_l.head(10)

,Gender,Total Bilirubin,Direct Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Albumin,Albumin_and_Globulin Ratio,Result
0,1,0.7,0.1,187.0,16.0,18.0,3.3,0.90,1
1,0,10.9,5.5,699.0,64.0,100.0,3.2,0.74,1
2,0,7.3,4.1,490.0,60.0,68.0,3.3,0.89,1
3,0,1.0,0.4,182.0,14.0,20.0,3.4,1.00,1
4,0,3.9,2.0,195.0,27.0,59.0,2.4,0.40,1
5,0,1.8,0.7,208.0,19.0,14.0,4.4,1.30,1
6,1,0.9,0.3,202.0,14.0,11.0,3.6,1.10,1
7,0,0.9,0.3,202.0,22.0,19.0,4.1,1.20,2
8,0,0.7,0.2,290.0,53.0,58.0,3.4,1.00,1
9,0,0.6,0.1,210.0,51.0,59.0,2.7,0.80,1


In [141]:
# Split the data into training and testing sets
X_diabetes, X_test_diabetes, y_diabetes, y_test_diabetes = train_test_split(new_data_d.drop('Outcome', axis=1), new_data_d['Outcome'], test_size=0.2, random_state=42)
X_heart, X_test_heart, y_heart, y_test_heart = train_test_split(new_data_h.drop('target', axis=1), new_data_h['target'], test_size=0.2, random_state=42)
X_kidney, X_test_kidney, y_kidney, y_test_kidney = train_test_split(new_data_k.drop('class', axis=1), new_data_k['class'], test_size=0.2, random_state=42)
X_liver, X_test_liver, y_liver, y_test_liver = train_test_split(new_data_l.drop('Result', axis=1), new_data_l['Result'], test_size=0.2, random_state=42)

In [142]:
# Create the individual models
diabetes_model = RandomForestClassifier()
heart_model = RandomForestClassifier()
kidney_model = RandomForestClassifier()
liver_model = RandomForestClassifier()

In [143]:
# Train the individual models
diabetes_model.fit(X_diabetes, y_diabetes)
heart_model.fit(X_heart, y_heart)
kidney_model.fit(X_kidney, y_kidney)
liver_model.fit(X_liver, y_liver)

RandomForestClassifier()

In [144]:
# Create the ensemble model
ensemble_model = VotingClassifier(estimators=[('diabetes', diabetes_model), ('heart', heart_model), ('kidney', kidney_model), ('liver', liver_model)], voting='hard')

In [145]:
# Train the ensemble model
ensemble_model.fit(np.concatenate((X_diabetes, X_heart, X_kidney, X_liver), axis=0), np.concatenate((y_diabetes, y_heart, y_kidney, y_liver), axis=0))

VotingClassifier(estimators=[('diabetes', RandomForestClassifier()),
                             ('heart', RandomForestClassifier()),
                             ('kidney', RandomForestClassifier()),
                             ('liver', RandomForestClassifier())])

In [146]:
# Evaluate the ensemble model
accuracy = accuracy_score(np.concatenate((y_test_diabetes, y_test_heart, y_test_kidney, y_test_liver), axis=0), ensemble_model.predict(np.concatenate((X_test_diabetes, X_test_heart, X_test_kidney, X_test_liver), axis=0)))
print('Accuracy:', accuracy)

Accuracy: 0.9949194113524877


In [147]:
import pickle

# Save the trained ensemble model to a file
with open('/content/ensemble_model.pkl', 'wb') as f:
    pickle.dump(ensemble_model, f)


In [148]:
# Load the saved model from file
import pickle
with open('ensemble_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [149]:
def predict_disease(disease_type, **kwargs):
    with open('ensemble_model.pkl', 'rb') as f:
        ensemble_model = pickle.load(f)

    if disease_type == 'heart':
        features = np.array([[kwargs.get('age'), kwargs.get('sex'), kwargs.get('cp'), kwargs.get('thalach'),
                              kwargs.get('exang'), kwargs.get('oldpeak'), kwargs.get('ca'), kwargs.get('thal')]])
    elif disease_type == 'diabetes':
        features = np.array([[kwargs.get('Pregnancies'), kwargs.get('Glucose'), kwargs.get('Insulin'), kwargs.get('BMI'),
                              kwargs.get('Age'), kwargs.get('NewInsulinScore_Normal'), kwargs.get('NewGlucose_Normal'),
                              kwargs.get('NewGlucose_Secret')]])
    elif disease_type == 'liver':
        features = np.array([[kwargs.get('Age'), kwargs.get('Gender'), kwargs.get('Total_Bilirubin'),
                              kwargs.get('Direct_Bilirubin'), kwargs.get('Alkaline_Phosphotase'),
                              kwargs.get('Alamine_Aminotransferase'), kwargs.get('Aspartate_Aminotransferase'),
                              kwargs.get('Albumin_and_Globulin_Ratio')]])
    elif disease_type == 'kidney':
        features = np.array([[kwargs.get('age'), kwargs.get('specific_gravity'), kwargs.get('albumin'),
                              kwargs.get('haemoglobin'), kwargs.get('packed_cell_volume'),
                              kwargs.get('hypertension'), kwargs.get('diabetes_mellitus'),
                              kwargs.get('coronary_artery_disease')]])
    else:
        return "Invalid disease type"

    prediction = ensemble_model.predict(features)
    return prediction[0]

# Call the function to predict heart disease
prediction = predict_disease(disease_type='heart', age=63, sex=1, cp=3, thalach=5.01, exang=0, oldpeak=2.3, ca=0, thal=1)
print("Predicted heart disease status:", prediction)

# Call the function to predict diabetes
prediction = predict_disease(disease_type='diabetes', Pregnancies=6, Glucose=148, Insulin=169, BMI=33.6, Age=50,
                             NewInsulinScore_Normal=0, NewGlucose_Normal=0, NewGlucose_Secret=1)
print("Predicted Diabetes disease status:", prediction)

# Call the function to predict liver disease
prediction = predict_disease(disease_type='liver', Age=65, Gender=1, Total_Bilirubin=0.7, Direct_Bilirubin=0.1,
                             Alkaline_Phosphotase=187, Alamine_Aminotransferase=16, Aspartate_Aminotransferase=18,
                             Albumin_and_Globulin_Ratio=0.90)
print("Predicted Liver disease status:", prediction)

# Call the function to predict kidney disease
prediction = predict_disease(disease_type='kidney', age=52, specific_gravity=1.025, albumin=0.0, haemoglobin=15,
                             packed_cell_volume=52, hypertension=0, diabetes_mellitus=0, coronary_artery_disease=0)
print("Predicted Kidney disease status:", prediction)


Predicted heart disease status: 0
Predicted Diabetes disease status: 1
Predicted Liver disease status: 1
Predicted Kidney disease status: 0
